In [1]:
!python -V

Python 3.9.16


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/alekseikatridi/my-mlops/2week/mlruns/1', creation_time=1685300379159, experiment_id='1', last_update_time=1685300379159, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715206931833

In [18]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [20]:
import xgboost as xgb

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[22:14:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:20.00166                                                                                                 
[1]	validation-rmse:18.89677                                                                                                 
[2]	validation-rmse:17.87316                                                                                                 
[3]	validation-rmse:16.92649                                                                                                 
[4]	validation-rmse:16.05115                                                                                                 
[5]	validation-rmse:15.24190                                                                                                 
[6]	validation-rmse:14.49534  

[63]	validation-rmse:6.59910                                                                                                 
[64]	validation-rmse:6.59491                                                                                                 
[65]	validation-rmse:6.59071                                                                                                 
[66]	validation-rmse:6.58690                                                                                                 
[67]	validation-rmse:6.58347                                                                                                 
[68]	validation-rmse:6.58044                                                                                                 
[69]	validation-rmse:6.57697                                                                                                 
[70]	validation-rmse:6.57394                                                                                          

[128]	validation-rmse:6.51112                                                                                                
[129]	validation-rmse:6.51062                                                                                                
[130]	validation-rmse:6.51029                                                                                                
[131]	validation-rmse:6.50997                                                                                                
[132]	validation-rmse:6.50974                                                                                                
[133]	validation-rmse:6.50940                                                                                                
[134]	validation-rmse:6.50916                                                                                                
[135]	validation-rmse:6.50884                                                                                         

[193]	validation-rmse:6.49787                                                                                                
[194]	validation-rmse:6.49786                                                                                                
[195]	validation-rmse:6.49777                                                                                                
[196]	validation-rmse:6.49764                                                                                                
[197]	validation-rmse:6.49748                                                                                                
[198]	validation-rmse:6.49739                                                                                                
[199]	validation-rmse:6.49716                                                                                                
[200]	validation-rmse:6.49712                                                                                         

[258]	validation-rmse:6.49117                                                                                                
[259]	validation-rmse:6.49086                                                                                                
[260]	validation-rmse:6.49081                                                                                                
[261]	validation-rmse:6.49053                                                                                                
[262]	validation-rmse:6.49047                                                                                                
[263]	validation-rmse:6.49040                                                                                                
[264]	validation-rmse:6.49024                                                                                                
[265]	validation-rmse:6.49022                                                                                         

...

[890]	validation-rmse:6.58058                                                                                                
[891]	validation-rmse:6.58063                                                                                                
[892]	validation-rmse:6.58062                                                                                                
[893]	validation-rmse:6.58059                                                                                                
[894]	validation-rmse:6.58045                                                                                                
[895]	validation-rmse:6.58045                                                                                                
[896]	validation-rmse:6.58044                                                                                                
[897]	validation-rmse:6.58044                                                                                         

[955]	validation-rmse:6.57926                                                                                                
[956]	validation-rmse:6.57928                                                                                                
[957]	validation-rmse:6.57923                                                                                                
[958]	validation-rmse:6.57919                                                                                                
[959]	validation-rmse:6.57919                                                                                                
[960]	validation-rmse:6.57920                                                                                                
[961]	validation-rmse:6.57921                                                                                                
[962]	validation-rmse:6.57920                                                                                         

[18]	validation-rmse:6.61025                                                                                                 
[19]	validation-rmse:6.60714                                                                                                 
[20]	validation-rmse:6.60619                                                                                                 
[21]	validation-rmse:6.60481                                                                                                 
[22]	validation-rmse:6.60376                                                                                                 
[23]	validation-rmse:6.60270                                                                                                 
[24]	validation-rmse:6.60150                                                                                                 
[25]	validation-rmse:6.60009                                                                                          

[83]	validation-rmse:6.56300                                                                                                 
[84]	validation-rmse:6.56284                                                                                                 
[85]	validation-rmse:6.56287                                                                                                 
[86]	validation-rmse:6.56265                                                                                                 
[87]	validation-rmse:6.56249                                                                                                 
[88]	validation-rmse:6.56239                                                                                                 
[89]	validation-rmse:6.56213                                                                                                 
[90]	validation-rmse:6.56206                                                                                          

[148]	validation-rmse:6.55753                                                                                                
[149]	validation-rmse:6.55764                                                                                                
[150]	validation-rmse:6.55706                                                                                                
[151]	validation-rmse:6.55677                                                                                                
[152]	validation-rmse:6.55693                                                                                                
[153]	validation-rmse:6.55682                                                                                                
[154]	validation-rmse:6.55680                                                                                                
[155]	validation-rmse:6.55666                                                                                         

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        